In [6]:
import numpy as np
from matplotlib import pyplot as plt
import os
import lecroyparser
import time


# mount my drive 
#from google.colab import drive
#drive.mount('/content/drive')
# path='/jupyter-workspace/cloud-storage/cygno-data/WC/'
def Gauss3(x, a0, x0, s0):
    import numpy as np
    return a0 * np.exp(-(x - x0)**2 / (2 * s0**2))

def UnderTh2Array(ArrayIn, Th):
    UnderTh    = False
    ThArr = []

    for i in range (0, len(ArrayIn)):
        if ArrayIn[i]<=Th and not UnderTh:
            UnderTh     = True
            ThArr.append([i, ArrayIn[i], UnderTh])
        if ArrayIn[i]>Th and UnderTh:
            UnderTh     = False
            ThArr.append([i, ArrayIn[i], UnderTh])
    return  ThArr

def wave(path, files, usamle, th = -0.005, x_cut_min = -8e-8, x_cut_max = -1e-8, 
         ax=False, verbose=False):
    LOG_EVERY_N = 100
    import numpy as np
    import lecroyparser
    # x_cut_min = -8e-8
    # x_cut_max = -1e-8
    under_th = []
    min_w = []
    int_w = []
    num_under_th = []
    min_pos_w = []
    for i, filename in enumerate(files):

        if (i % LOG_EVERY_N) == 0:
            print("log", i, filename)
        f = open(path+filename, 'rb')
        ntry = 3
        while ntry>0:
            try:
                if usamle:
                    data = lecroyparser.ScopeData(data = f.read(), sparse = usamle)
                else:
                    data = lecroyparser.ScopeData(data = f.read())
                ntry=0
            except:
                print ("reading data error "+filename+", retray", ntry)
                time.sleep(1)
                ntry -=1
        data_offset = np.mean(data.y[data.x<x_cut_min])
        data_offsigma = np.std(data.y[data.x<x_cut_min])
        if verbose: 
            print(i, path, filename)
            print(data_offset, data_offsigma)
            print(x_cut_min, x_cut_max)
        #print(data_offset, data_offsigma)
        y = np.array(data.y-data_offset)
        min_w.append(np.min(y))
        int_w.append(np.trapz(y[(data.x>x_cut_min) & (data.x<x_cut_max)]))
        #int_w.append(np.trapz(y))
        #min_pos_w.append(data.x[np.where(data.y == data.y.min())][0])
        under_th.append(filename)

        if min(y)<th:      
            min_array = UnderTh2Array(y, th)
            min_pos_w.append(data.x[np.where(y < th)][0])
            num_under_th.append(len(min_array))
        else:
            min_pos_w.append(-1)
            num_under_th.append(0)
        if ax:
            ax.plot(data.x, y, 
                    label="min: {:.2e}\n pos: {:.2e}\n int: {:.2e}".format(min_w[0], 
                                                                           min_pos_w[0], 
                                                                           int_w[0]))
            print(num_under_th)

            
        if verbose: print (data)
        f.close()
    if ax:
        #plt.ylim(-0.030, 0.010)
        ax.axhline(th)
        ax.axvline(x_cut_min)
        ax.axvline(x_cut_max)
        ax.axhline(th)
        ax.legend()
        
    
    return np.array(under_th), np.array(num_under_th), np.array(int_w), np.array(min_pos_w), ax


def histGaus(var, bins, ax, color='b', xrange=False, alpha=1, label=''):
    from sklearn.metrics import r2_score
    from scipy.stats import chisquare
    import scipy.stats as stats
    from scipy.optimize import curve_fit
    import numpy as np
    var = np.array(var)
    if xrange:
        ax.hist(var, bins=bins, label=label, color=color, range=(xrange[0], xrange[1]), alpha=alpha)
        y, bins_edge = np.histogram(var, bins=bins, range=(xrange[0], xrange[1]))
        p0=[y.max(),bins_edge[y.argmax()], np.std(var[var>xrange[0]])]
        #print(p0)
    else:
        ax.hist(var, bins=bins, label=label, color=color)
        y, bins_edge = np.histogram(var, bins=bins)
        p0=[y.max(),bins_edge[y.argmax()], np.std(var)]
    x = np.linspace(bins_edge[0], bins_edge[-1], bins)
    popt, pcov = curve_fit(Gauss3, x, y, p0=p0)
    perr = np.sqrt(np.diag(pcov))
    r2=r2_score(y, Gauss3(x, *popt))
    x = np.linspace(bins_edge[0], bins_edge[-1], 100)
    sstat='a = {0:.2e} +/- {1:.2e}\nb = {2:.2e} +/- {3:.2e} \
    \nc = {4:.2e} +/- {5:.2e}\n$R^2$ = {6:.4f}'.format(popt[0], perr[0], popt[1], perr[1], 
                                                popt[2], perr[2], r2)
    ax.plot(x,Gauss3(x, *popt), color+'--', label=sstat)
    return ax

def outdata(C1, C2, C3, C4, bins, title='no title'):
    fig, ax = plt.subplots(1,4, figsize=(16, 4))
    ax[0].hist(C1, bins=bins, color='r', 
               label="N={:d}\nm={:.2e}\ns={:.2e}".format(len(C1), np.mean(C1), np.std(C1)))
    ax[0].set_title("C1")
    # ax[0].set_xlim(-0.1, 0)
    ax[0].legend()
    ax[0].set_yscale('log')
    ax[1].hist(C2, bins=bins, color='gray', 
               label="N={:d}\nm={:.2e}\ns={:.2e}".format(len(C2), np.mean(C2), np.std(C2)))
    ax[1].set_title("C2")
    ax[1].set_yscale('log')
    ax[1].legend()
    ax[2].hist(C3, bins=bins, color='b', 
               label="N={:d}\nm={:.2e}\ns={:.2e}".format(len(C3), np.mean(C3), np.std(C3)))
    ax[2].set_title("C3")
    ax[2].set_yscale('log')
    ax[2].legend()
    ax[3].hist(C4, bins=bins, color='g', 
              label="N={:d}\nm={:.2e}\ns={:.2e}".format(len(C4), np.mean(C4), np.std(C4)))
    ax[3].set_title("C4")
    ax[3].set_yscale('log')
    ax[3].legend()
    # ax[1].set_xlim(-50, 0)
    # ax[1].set_yscale('log')
    # ax[2]=histGaus(min_pos_w, bins=bins, ax=ax[2], color='b', alpha=1, label='')
    # ax[2].set_title("time th")
    fig.suptitle(title)
    plt.show()
    return


def outdata2(C1, C2, th1, th2, bins, title='no title', verbose=False):
    C1 = np.array(C1)
    C2 = np.array(C2)
    print (len(C1), len(C2))
    fig, ax = plt.subplots(1,4, figsize=(16, 4))
    cond=(C1<th1) & (C2<th2)
    #print(cond)
    ax[0].plot(C1[cond], C2[cond], 'k.')
    ax[0].set_title("C1 vs C2 (C1<{:.3f} C2<{:.3f})".format(th1, th2))
    # ax[0].set_xlim(-20, 0)
    # ax[0].set_ylim(-20, 0)
    ax[1].hist2d(C1[cond], C2[cond], 
           bins=(40,40), cmap=plt.cm.jet)
    # ax[1].set_xlim(-80, 0)
    # ax[1].set_ylim(-80, 0)
    # ax[1]=histGaus(C1[cond]/C2[cond], bins=50, ax=ax[1], color='b', label="{:.2f}".format(np.mean(C1[cond]/C2[cond])))
    ax[2].hist(C1[cond]/C2[cond], bins, color='gray', label="{:.2f}".format(np.mean(C1[cond]/C2[cond])))
    ax[2].set_title("C1/C2 (C1<{:.3f} C2<{:.3f})".format(th1, th2))
    #ax[2].set_xlim(0, 1000)
    ax[2].set_yscale("log")
    ax[2].legend()
    ax[3].hist(C1[cond]-C2[cond], bins, color='gray', label="{:.2f}".format(np.mean(C1[cond]/C2[cond])))
    ax[3].set_title("C1-C2 (C1<{:.3f} C2<{:.3f})".format(th1, th2))
    #ax[3].set_yscale("log")
    ax[3].legend()
    fig.suptitle(title)
    plt.show()
    return


